In [1]:
!pip install sentence-transformers pandas scikit-learn

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
!pip install nltk
# !pip install spacy
# !pip install wordcloud
# !pip install textblob
# !pip install pycaret[classification]
# !pip install gensim
# !pip install sentence-transformers
# !pip install scikit-learn==0.23.2
# !pip install pycaret==2.3.10

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import re
import nltk
# import spacy
from nltk.corpus import stopwords
# from wordcloud import WordCloud
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import plotly.express as px
# from textblob import TextBlob

from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,roc_curve,roc_auc_score,auc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb

In [5]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
url = "https://raw.githubusercontent.com/noequalindi/1000IA_course/main/cyberbullying_tweets.csv"
df_raw = pd.read_csv(url, sep=',')
df_raw.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [8]:
# Define la función clean_tweet
def clean_tweet(tweet:str) -> str:
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE) # Remove urls
    tweet = re.sub(r'@[A-Za-z0-9]+','', tweet) # Remove @mentions
    tweet = re.sub(r'\#[A-Za-z0-9]+','', tweet) # Remove hashtags
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet) # Remove special characters and punctuation
    tweet = tweet.lower() # Convert to lowercase

    tokens = word_tokenize(tweet) # Tokenize

    stop_words = set(stopwords.words('english')) # Remove stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    clean_tweet = ' '.join(filtered_tokens) # Join tokens back into string

    return clean_tweet

def preprocess_text(df, col_name):
    #  Convertir texto a mayúsculas
    df[col_name] = df[col_name].str.upper()

    # Eliminar URLs, menciones, hashtags y caracteres especiales
    df[col_name] = df[col_name].apply(clean_tweet)

    # Trim
    df[col_name] = df[col_name].str.strip()

    # Tokenización
    df['tokens'] = df[col_name].apply(word_tokenize)

    # Stemming
    stemmer = PorterStemmer()
    df['stemmed'] = df['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])
    df['stemmed'] = df['stemmed'].apply(lambda x: ' '.join(x))
    # Lematización
    lemmatizer = WordNetLemmatizer()
    df['lemmatized'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    df['lemmatized'] = df['lemmatized'].apply(lambda x: ' '.join(x))

    return df


In [9]:
preprocessed_df = preprocess_text(df_raw, 'tweet_text')

In [10]:
preprocessed_df

,tweet_text,cyberbullying_type,tokens,stemmed,lemmatized
0,words food crapilicious,not_cyberbullying,"[words, food, crapilicious]",word food crapilici,word food crapilicious
1,white,not_cyberbullying,[white],white,white
2,classy whore red velvet cupcakes,not_cyberbullying,"[classy, whore, red, velvet, cupcakes]",classi whore red velvet cupcak,classy whore red velvet cupcake
3,gio meh p thanks heads concerned another angry...,not_cyberbullying,"[gio, meh, p, thanks, heads, concerned, anothe...",gio meh p thank head concern anoth angri dude ...,gio meh p thanks head concerned another angry ...
4,isis account pretending kurdish account like i...,not_cyberbullying,"[isis, account, pretending, kurdish, account, ...",isi account pretend kurdish account like islam...,isi account pretending kurdish account like is...
...,...,...,...,...,...
47687,black ppl arent expected anything depended any...,ethnicity,"[black, ppl, arent, expected, anything, depend...",black ppl arent expect anyth depend anyth yet ...,black ppl arent expected anything depended any...
47688,turner withhold disappointment turner called c...,ethnicity,"[turner, withhold, disappointment, turner, cal...",turner withhold disappoint turner call court a...,turner withhold disappointment turner called c...
47689,swear god dumb nigger bitch got bleach hair re...,ethnicity,"[swear, god, dumb, nigger, bitch, got, bleach,...",swear god dumb nigger bitch got bleach hair re...,swear god dumb nigger bitch got bleach hair re...
47690,yea fuck rt youre nigger fucking unfollow fuck...,ethnicity,"[yea, fuck, rt, youre, nigger, fucking, unfoll...",yea fuck rt your nigger fuck unfollow fuck dum...,yea fuck rt youre nigger fucking unfollow fuck...


# Sentence Transformer

In [11]:
textos = preprocessed_df['tweet_text'].tolist()
targets = preprocessed_df['cyberbullying_type'].tolist()

In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')
texto_vectores = model.encode(textos)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(texto_vectores, targets, test_size=0.2, random_state=42)

In [17]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print("RandomForestClassifier - Reporte de Clasificación:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

RandomForestClassifier - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.90      0.94      0.92      1603
          ethnicity       0.95      0.86      0.90      1603
             gender       0.83      0.78      0.81      1531
  not_cyberbullying       0.46      0.47      0.47      1624
other_cyberbullying       0.45      0.47      0.46      1612
           religion       0.90      0.94      0.92      1566

           accuracy                           0.74      9539
          macro avg       0.75      0.74      0.75      9539
       weighted avg       0.75      0.74      0.74      9539



In [18]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print("SVM - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_svc))

SVM - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.97      0.97      1603
          ethnicity       0.97      0.94      0.95      1603
             gender       0.88      0.83      0.85      1531
  not_cyberbullying       0.60      0.54      0.56      1624
other_cyberbullying       0.58      0.68      0.62      1612
           religion       0.94      0.95      0.95      1566

           accuracy                           0.81      9539
          macro avg       0.82      0.82      0.82      9539
       weighted avg       0.82      0.81      0.82      9539



In [19]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.95      0.96      0.96      1603
          ethnicity       0.95      0.93      0.94      1603
             gender       0.85      0.81      0.83      1531
  not_cyberbullying       0.56      0.52      0.54      1624
other_cyberbullying       0.58      0.63      0.60      1612
           religion       0.92      0.95      0.93      1566

           accuracy                           0.80      9539
          macro avg       0.80      0.80      0.80      9539
       weighted avg       0.80      0.80      0.80      9539



In [20]:
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
y_pred_gbc = gbc.predict(X_test)
print("Gradient Boosting - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_gbc))

Gradient Boosting - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.93      0.94      0.93      1603
          ethnicity       0.94      0.88      0.91      1603
             gender       0.83      0.80      0.81      1531
  not_cyberbullying       0.52      0.49      0.50      1624
other_cyberbullying       0.53      0.59      0.55      1612
           religion       0.91      0.94      0.92      1566

           accuracy                           0.77      9539
          macro avg       0.78      0.77      0.77      9539
       weighted avg       0.77      0.77      0.77      9539



In [21]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("K-Nearest Neighbors - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbors - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.84      0.98      0.91      1603
          ethnicity       0.88      0.93      0.90      1603
             gender       0.77      0.86      0.82      1531
  not_cyberbullying       0.58      0.48      0.52      1624
other_cyberbullying       0.59      0.48      0.53      1612
           religion       0.92      0.95      0.94      1566

           accuracy                           0.78      9539
          macro avg       0.76      0.78      0.77      9539
       weighted avg       0.76      0.78      0.77      9539



In [24]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Decision Tree - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_dt))

Decision Tree - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.80      0.77      0.78      1603
          ethnicity       0.73      0.70      0.71      1603
             gender       0.61      0.65      0.63      1531
  not_cyberbullying       0.34      0.36      0.35      1624
other_cyberbullying       0.32      0.30      0.31      1612
           religion       0.78      0.77      0.78      1566

           accuracy                           0.59      9539
          macro avg       0.60      0.59      0.59      9539
       weighted avg       0.59      0.59      0.59      9539



# TDIDF

In [13]:
vectorizer = TfidfVectorizer()
texto_vectores = vectorizer.fit_transform(textos)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(texto_vectores, targets, test_size=0.2, random_state=42)

In [15]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print("RandomForestClassifier - TDIDF - Reporte de Clasificación:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

RandomForestClassifier - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.97      0.98      0.97      1603
          ethnicity       0.98      0.98      0.98      1603
             gender       0.87      0.81      0.84      1531
  not_cyberbullying       0.53      0.47      0.50      1624
other_cyberbullying       0.52      0.62      0.57      1612
           religion       0.94      0.95      0.95      1566

           accuracy                           0.80      9539
          macro avg       0.80      0.80      0.80      9539
       weighted avg       0.80      0.80      0.80      9539



In [16]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print("SVM - TDIDF - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_svc))

SVM - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.96      0.96      1603
          ethnicity       0.98      0.96      0.97      1603
             gender       0.91      0.79      0.84      1531
  not_cyberbullying       0.53      0.55      0.54      1624
other_cyberbullying       0.55      0.62      0.58      1612
           religion       0.95      0.92      0.94      1566

           accuracy                           0.80      9539
          macro avg       0.81      0.80      0.81      9539
       weighted avg       0.81      0.80      0.80      9539



In [17]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression - TDIDF - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.96      0.96      1603
          ethnicity       0.98      0.96      0.97      1603
             gender       0.90      0.80      0.85      1531
  not_cyberbullying       0.53      0.56      0.54      1624
other_cyberbullying       0.56      0.61      0.59      1612
           religion       0.94      0.93      0.94      1566

           accuracy                           0.80      9539
          macro avg       0.81      0.80      0.81      9539
       weighted avg       0.81      0.80      0.80      9539



In [20]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("K-Nearest Neighbors - TDIDF - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbors - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.08      0.15      1603
          ethnicity       0.97      0.24      0.39      1603
             gender       0.18      0.98      0.30      1531
  not_cyberbullying       0.51      0.08      0.14      1624
other_cyberbullying       0.43      0.05      0.09      1612
           religion       0.95      0.04      0.07      1566

           accuracy                           0.24      9539
          macro avg       0.67      0.24      0.19      9539
       weighted avg       0.67      0.24      0.19      9539



In [21]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("Naive Bayes - TDIDF - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_nb))

Naive Bayes - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.69      0.98      0.81      1603
          ethnicity       0.81      0.91      0.86      1603
             gender       0.76      0.83      0.80      1531
  not_cyberbullying       0.64      0.32      0.43      1624
other_cyberbullying       0.60      0.38      0.47      1612
           religion       0.77      0.97      0.86      1566

           accuracy                           0.73      9539
          macro avg       0.71      0.73      0.70      9539
       weighted avg       0.71      0.73      0.70      9539



In [22]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Decision Tree - TDIDF - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_dt))

Decision Tree - TDIDF - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.98      0.95      0.97      1603
          ethnicity       0.97      0.96      0.97      1603
             gender       0.82      0.80      0.81      1531
  not_cyberbullying       0.47      0.47      0.47      1624
other_cyberbullying       0.49      0.53      0.51      1612
           religion       0.94      0.93      0.93      1566

           accuracy                           0.77      9539
          macro avg       0.78      0.77      0.78      9539
       weighted avg       0.78      0.77      0.77      9539



# TF-IDF con n-grams

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
texto_vectores = vectorizer.fit_transform(textos)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(texto_vectores, targets, test_size=0.2, random_state=42)

In [ ]:
# clf = RandomForestClassifier()
# clf.fit(X_train, y_train)
# print("RandomForestClassifier - TDIDF y n-grams - Reporte de Clasificación:")
# y_pred = clf.predict(X_test)
# print(classification_report(y_test, y_pred))

In [15]:
svc = SVC()
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print("SVM - TDIDF y n-grams - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_svc))

SVM - TDIDF y n-grams - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.95      0.95      1603
          ethnicity       0.98      0.93      0.96      1603
             gender       0.92      0.73      0.82      1531
  not_cyberbullying       0.50      0.53      0.51      1624
other_cyberbullying       0.49      0.62      0.55      1612
           religion       0.95      0.91      0.93      1566

           accuracy                           0.78      9539
          macro avg       0.80      0.78      0.79      9539
       weighted avg       0.80      0.78      0.78      9539



In [16]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression - TDIDF y n-grams - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_lr))

Logistic Regression - TDIDF y n-grams - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.96      0.96      0.96      1603
          ethnicity       0.98      0.95      0.97      1603
             gender       0.91      0.77      0.83      1531
  not_cyberbullying       0.50      0.55      0.52      1624
other_cyberbullying       0.52      0.56      0.54      1612
           religion       0.94      0.93      0.93      1566

           accuracy                           0.79      9539
          macro avg       0.80      0.79      0.79      9539
       weighted avg       0.80      0.79      0.79      9539



In [17]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
print("K-Nearest Neighbors - TDIDF y n-grams - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))

K-Nearest Neighbors - TDIDF y n-grams - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       1.00      0.03      0.06      1603
          ethnicity       1.00      0.10      0.19      1603
             gender       0.17      0.99      0.28      1531
  not_cyberbullying       0.63      0.02      0.04      1624
other_cyberbullying       0.49      0.01      0.03      1612
           religion       1.00      0.01      0.03      1566

           accuracy                           0.19      9539
          macro avg       0.71      0.20      0.10      9539
       weighted avg       0.72      0.19      0.10      9539



In [18]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("Naive Bayes - TDIDF y n-grams - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_nb))

Naive Bayes - TDIDF y n-grams - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.73      0.99      0.84      1603
          ethnicity       0.84      0.94      0.89      1603
             gender       0.74      0.84      0.79      1531
  not_cyberbullying       0.56      0.31      0.40      1624
other_cyberbullying       0.54      0.33      0.41      1612
           religion       0.78      0.98      0.87      1566

           accuracy                           0.73      9539
          macro avg       0.70      0.73      0.70      9539
       weighted avg       0.70      0.73      0.70      9539



In [19]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
print("Decision Tree - TDIDF y n-grams - Reporte de Clasificación:")
print(classification_report(y_test, y_pred_dt))

Decision Tree - TDIDF y n-grams - Reporte de Clasificación:
                     precision    recall  f1-score   support

                age       0.98      0.96      0.97      1603
          ethnicity       0.98      0.96      0.97      1603
             gender       0.84      0.81      0.82      1531
  not_cyberbullying       0.47      0.47      0.47      1624
other_cyberbullying       0.48      0.51      0.50      1612
           religion       0.94      0.93      0.94      1566

           accuracy                           0.77      9539
          macro avg       0.78      0.77      0.78      9539
       weighted avg       0.78      0.77      0.78      9539

